<a href="https://colab.research.google.com/github/MingmaG/GEN_AI/blob/main/vector_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb langchain-community
!pip install langchain-community langchain_openai langchain-core
!pip install huggingface_hub
!pip install transformers sentence-transformers


In [ ]:
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

hf_token = "hf_FWIIISLWKuMNOABEDxzwjQSkpCEbURam"
login(hf_token)

In [ ]:
model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
sentences = [
    'what is your name',
    'I am Nepali'
]
embedded = model.embed_documents(sentences)
print(embedded)


[[-0.07185039669275284, 0.03329799324274063, 0.02547474578022957, 0.02628723531961441, -0.05714616924524307, -0.04249054566025734, 0.17539693415164948, -0.022255273535847664, 0.06143518537282944, 0.005702488124370575, -0.04874107241630554, -0.12996812164783478, 0.04499742388725281, -0.034108374267816544, -0.03242511302232742, -0.011028178967535496, 0.024463780224323273, 0.03130440413951874, -0.10177114605903625, -0.0880526527762413, -0.10171739012002945, 0.03881379961967468, -0.05878493934869766, -0.01770128682255745, -0.04726884514093399, 0.03541528433561325, -0.029594706371426582, 0.0500899963080883, -0.025855228304862976, -0.12366403639316559, 0.04011086747050285, -0.05338606238365173, 0.10339538007974625, 0.06660550832748413, 0.00018430559430271387, -0.04693657532334328, -0.11863788217306137, 0.00955781526863575, -0.019044941291213036, -0.0064897118136286736, -0.019765721634030342, -0.11091035604476929, -0.020859630778431892, -0.011532405391335487, -0.0021487330086529255, 0.0297480

In [ ]:
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
docs = [
    Document(
        page_content="Mingmar Gyaltsen is a student of electronic and communication in Nepal",
             metadata={}),
    Document(
        page_content="Lionel Messi is an Argentine professional footballer who plays as a forward.",
        metadata={"name": "Lionel Messi", "position": "Forward", "country": "Argentina"}
    ),
    Document(
        page_content="Cristiano Ronaldo is a Portuguese professional footballer who plays as a forward.",
        metadata={"name": "Cristiano Ronaldo", "position": "Forward", "country": "Portugal"}
    ),
    Document(
        page_content="Neymar Jr. is a Brazilian professional footballer who plays as a forward.",
        metadata={"name": "Neymar Jr.", "position": "Forward", "country": "Brazil"}
    ),
    Document(
        page_content="Kylian Mbappé is a French professional footballer who plays as a forward.",
        metadata={"name": "Kylian Mbappé", "position": "Forward", "country": "France"}
    ),
    Document(
        page_content="Luka Modrić is a Croatian professional footballer who plays as a midfielder.",
        metadata={"name": "Luka Modrić", "position": "Midfielder", "country": "Croatia"}
    )
]


In [ ]:
vector_store = Chroma(
    embedding_function= model,
    persist_directory='chroma_db',
    collection_name="sample"
)

In [ ]:
vector_store.add_documents(docs)

['67bdf38c-6359-4a57-884d-eb96473db404',
 'c7cbfa79-6187-4d73-a3fd-dd9f8785ace6',
 'c7f793ac-5099-4779-a252-40dd2367be56',
 '1637d54b-49b9-411d-95c3-923f0e0618e7',
 '9bdaecc1-367f-46ff-b8a0-0988920c2777',
 'a864bb83-f780-42d7-95b8-5941728c4437']

In [ ]:
#1 similarity_search in vector store
results = vector_store.similarity_search('who is the player from france', k=3)
for i in results:
  print(i.page_content)

Kylian Mbappé is a French professional footballer who plays as a forward.
Kylian Mbappé is a French professional footballer who plays as a forward.
Neymar Jr. is a Brazilian professional footballer who plays as a forward.


In [ ]:
# 2 similarity search with score
results2 = vector_store.similarity_search_with_score('Messi from argentina',k=3)
for doc, score in results2:
  print(score , doc.page_content)

0.5094665288925171 Lionel Messi is an Argentine professional footballer who plays as a forward.
0.5094665288925171 Lionel Messi is an Argentine professional footballer who plays as a forward.
1.1923123598098755 Cristiano Ronaldo is a Portuguese professional footballer who plays as a forward.


In [ ]:
# 3 MMR (maximal Marginal Relevance) diverse still relecant documents
results3 = vector_store.max_marginal_relevance_search("Best player in the Nepal", k =2)
results3

[Document(metadata={}, page_content='Bobby is a football player from india is also a best player'),
 Document(metadata={}, page_content='Mingmar Gyaltsen is a student of electronic and communication in Nepal')]

In [ ]:
# 4 add in vector store
new_docs = [Document(page_content ="Bobby is a football player from india is also a best player")]
vector_store.add_documents(new_docs)

['19f54d06-f1c2-460c-b6cd-302d847b66ef']

In [ ]:
vector_store.delete(ids=['doc1'])

In [ ]:
#5 Use as a Retriever
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={"k":3})
retriever.get_relevant_documents("Best player")

[Document(metadata={}, page_content='Bobby is a football player from india is also a best player'),
 Document(metadata={'country': 'Argentina', 'name': 'Lionel Messi', 'position': 'Forward'}, page_content='Lionel Messi is an Argentine professional footballer who plays as a forward.'),
 Document(metadata={'name': 'Lionel Messi', 'country': 'Argentina', 'position': 'Forward'}, page_content='Lionel Messi is an Argentine professional footballer who plays as a forward.')]

In [ ]:
data = vector_store.get()
print(data.keys())

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])


In [ ]:
#5 count documents
count = vector_store._collection.count()
count

13

In [ ]:
#6 get all docs and metadata
vector_store.get()

{'ids': ['02ee93c3-1870-4c52-9a84-9096197bfe0b',
  'a12840aa-9bd4-410b-8fa6-5b78cb7d9b15',
  '70932c07-b1d7-4fe6-a639-3520443d856c',
  'a5b671d3-d8e2-4a08-b421-b41a9d8bed1b',
  'f32fd345-46a7-4e0d-904f-a10b2b0ceae3',
  'af014d68-e1f8-4617-86f7-9de12a7e3785',
  '19f54d06-f1c2-460c-b6cd-302d847b66ef',
  'c7cbfa79-6187-4d73-a3fd-dd9f8785ace6',
  'c7f793ac-5099-4779-a252-40dd2367be56',
  '1637d54b-49b9-411d-95c3-923f0e0618e7',
  '9bdaecc1-367f-46ff-b8a0-0988920c2777',
  'a864bb83-f780-42d7-95b8-5941728c4437',
  '67bdf38c-6359-4a57-884d-eb96473db404'],
 'embeddings': None,
 'documents': ['Lionel Messi is an Argentine professional footballer who plays as a forward.',
  'Cristiano Ronaldo is a Portuguese professional footballer who plays as a forward.',
  'Neymar Jr. is a Brazilian professional footballer who plays as a forward.',
  'Kylian Mbappé is a French professional footballer who plays as a forward.',
  'Luka Modrić is a Croatian professional footballer who plays as a midfielder.',
  '